In [20]:
from IPython.display import display, Markdown
import os
from LS_AMG_RAG.data_snythesis import prompt_utils
import google.generativeai as palm

import chromadb
import chromadb.utils.embedding_functions as embedding_functions
chroma_client = chromadb.Client()
gemini = prompt_utils.Gemini()
google_ef  = embedding_functions.GoogleGenerativeAiEmbeddingFunction(api_key=os.environ['GEMINI_API_KEY'])

In [6]:
documents = []
metadata = []
ids = []
for idx, file_name in enumerate(os.listdir("../data/business_docs")):
    with open(f"../data/business_docs/{file_name}", "r") as f:
        file_contents = f.read()
        documents.append(file_contents)
        metadata.append({
            "type": "business document",
        })
        ids.append(f"business_doc_{idx}")

In [13]:
for m in palm.list_models():
    print(m.name)

models/chat-bison-001
models/text-bison-001
models/embedding-gecko-001
models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-pro
models/gemini-pro-vision
models/embedding-001
models/aqa


In [21]:
collection = chroma_client.create_collection(name="my_collection", embedding_function=google_ef)

In [22]:
collection.add(
    documents=documents,
    metadatas=metadata,
    ids=ids,
)

In [23]:
import json
print(json.dumps(collection.get(), indent=2))

{
  "ids": [
    "business_doc_0",
    "business_doc_1",
    "business_doc_2"
  ],
  "embeddings": null,
  "metadatas": [
    {
      "type": "business document"
    },
    {
      "type": "business document"
    },
    {
      "type": "business document"
    }
  ],
  "documents": [
    "\n### Company Name: Instagram\n### Category of Document: Business Proposal\n\n---\n\n#### **Business Proposal 1: Social Media Marketing Services**\n\n**Executive Summary:**\nInstagram offers comprehensive social media marketing services tailored to enhance brand presence and engagement. With a team of experienced digital marketers, we aim to elevate your social media strategy and maximize audience reach.\n\n**Company Overview:**\nInstagram, a leader in social media platforms, leverages its expertise and innovative tools to deliver impactful marketing solutions. Our focus on visual content and community engagement sets us apart in the industry.\n\n**Problem Statement:**\nMany businesses struggle to effe

In [24]:
queries = [
    "What is the status of the Direct Messaging update? Also, what are the challenges that we are facing?",
]

results = collection.query(
    query_texts=queries,
    n_results=2
)

In [32]:
results

{'ids': [['business_doc_2', 'business_doc_0']],
 'distances': [[0.7423908114433289, 0.5049270391464233]],
 'metadatas': [[{'type': 'business document'}, {'type': 'business document'}]],
 'embeddings': None,
 'documents': [['# Instagram Progress Report\n\n| Project Name   | Status      | Achievements                   | Challenges                  | Action Plans                         | Resource Allocation | Financial Updates |\n|-----------------|-------------|---------------------------------|-----------------------------|--------------------------------------|---------------------|-------------------|\n| Feed Redesign   | In Progress | Redesigned layout and UI       | Technical issues            | Conduct bug fixes and testing        | 5 developers       | $10,000           |\n| Stories Upgrade | Completed   | Added new interactive elements | User feedback for improvements| Implement new features based on feedback | 3 designers, 2 developers | $5,000 |\n| Direct Messaging Update | O

In [36]:
results.items()

dict_items([('ids', [['business_doc_2', 'business_doc_0']]), ('distances', [[0.7423908114433289, 0.5049270391464233]]), ('metadatas', [[{'type': 'business document'}, {'type': 'business document'}]]), ('embeddings', None), ('documents', [['# Instagram Progress Report\n\n| Project Name   | Status      | Achievements                   | Challenges                  | Action Plans                         | Resource Allocation | Financial Updates |\n|-----------------|-------------|---------------------------------|-----------------------------|--------------------------------------|---------------------|-------------------|\n| Feed Redesign   | In Progress | Redesigned layout and UI       | Technical issues            | Conduct bug fixes and testing        | 5 developers       | $10,000           |\n| Stories Upgrade | Completed   | Added new interactive elements | User feedback for improvements| Implement new features based on feedback | 3 designers, 2 developers | $5,000 |\n| Direct Mess

In [33]:
sorted(results.items(), key=lambda x: x[1], reverse=True)

KeyError: 0

In [60]:
results['documents'][0][0]

"\n### Company Name: Instagram\n### Category of Document: Business Proposal\n\n---\n\n#### **Business Proposal 1: Social Media Marketing Services**\n\n**Executive Summary:**\nInstagram offers comprehensive social media marketing services tailored to enhance brand presence and engagement. With a team of experienced digital marketers, we aim to elevate your social media strategy and maximize audience reach.\n\n**Company Overview:**\nInstagram, a leader in social media platforms, leverages its expertise and innovative tools to deliver impactful marketing solutions. Our focus on visual content and community engagement sets us apart in the industry.\n\n**Problem Statement:**\nMany businesses struggle to effectively utilize social media for brand promotion and customer interaction, leading to missed opportunities for growth and visibility.\n\n**Proposed Solution:**\nOur social media marketing services encompass strategic content creation, optimized ad campaigns, targeted audience analysis, a

In [61]:
metaprompt = """You are a helpful and informative bot that answers questions using text from the reference passage included below. \
  Be sure to respond in a complete sentence, being comprehensive, including all relevant background information. \
  However, you are talking to a non-technical audience, so be sure to break down complicated concepts and \
  strike a friendly and converstional tone. \
  If the passage is irrelevant to the answer, you may ignore it.
  QUESTION: '{query}'
  PASSAGE: '{relevant_passage}'

  ANSWER:
"""

gemini_result = gemini.send_message(message=metaprompt.format(query=queries[0], relevant_passage=results['documents'][0][0])).text
display(Markdown(gemini_result))

I'm sorry, but the provided passage does not contain any information regarding the status of the Direct Messaging update and the challenges faced by the team. Therefore, I cannot answer your question based on the provided information.

In [26]:
results['documents'][0][1]

'# Instagram Progress Report\n\n| Project Name   | Status      | Achievements                   | Challenges                  | Action Plans                         | Resource Allocation | Financial Updates |\n|-----------------|-------------|---------------------------------|-----------------------------|--------------------------------------|---------------------|-------------------|\n| Feed Redesign   | In Progress | Redesigned layout and UI       | Technical issues            | Conduct bug fixes and testing        | 5 developers       | $10,000           |\n| Stories Upgrade | Completed   | Added new interactive elements | User feedback for improvements| Implement new features based on feedback | 3 designers, 2 developers | $5,000 |\n| Direct Messaging Update | On Hold  | Enhanced encryption features  | Integration issues          | Collaborate with security team for solutions | -                  | -              |\n| Explore Algorithm Enhancement | In Progress | Improved personal

In [57]:
metaprompt = """You are a helpful and informative bot that answers questions using text from the reference passage included below. \
  Be sure to respond in a complete sentence, being comprehensive, including all relevant background information. \
  However, you are talking to a non-technical audience, so be sure to break down complicated concepts and \
  strike a friendly and converstional tone. \
  If the passage is irrelevant to the answer, you may ignore it.
  QUESTION: '{query}'
  PASSAGE: '{relevant_passage}'

  ANSWER:
"""

gemini_result = gemini.send_message(message=metaprompt.format(query=queries[0], relevant_passage=results['documents'][0][1]))

In [59]:
gemini_result.text

'Currently, the Direct Messaging update is on hold due to integration issues. The team is collaborating with the security team to find solutions to these issues.'